# Chapter 12: Facial Recognition & Landmark Detection (OpenCV DNN + Dlib)

## Objective
To implement face detection, recognition using face embeddings, and facial landmark detection using OpenCV's DNN module and Dlib. This lab provides a foundation for building real-time face-based systems.


## 1. What is Facial Recognition?

**Description**: Facial recognition involves identifying or verifying a person's identity from an image. It often includes face detection, alignment, embedding extraction, and matching.

### Key Components:
- **Face Detection**: Locating faces in images
- **Face Alignment**: Normalizing face orientation
- **Feature Extraction**: Creating face embeddings
- **Face Matching**: Comparing embeddings for recognition


## 2. Environment Setup

### Required Libraries:
- **OpenCV** for face detection and image processing
- **Dlib** for landmarks and face embeddings
- **face_recognition** wrapper for simplified face encoding and comparison

### Installation:
```bash
pip install opencv-python dlib face_recognition imutils numpy
```


In [ ]:
# Import required libraries
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
from pathlib import Path

# Try to import face_recognition and dlib
try:
    import face_recognition
    import dlib
    print("✅ All libraries imported successfully!")
    print(f"OpenCV version: {cv2.__version__}")
    print(f"Dlib version: {dlib.version}")
except ImportError as e:
    print(f"❌ Import error: {e}")
    print("Please install missing libraries:")
    print("pip install face-recognition dlib")


## 3. Face Detection using OpenCV DNN

We'll use OpenCV's DNN module with a pre-trained face detection model for robust face detection.


In [ ]:
def load_face_detector():
    """
    Load OpenCV DNN face detection model
    Using OpenCV's built-in face detection model
    """
    try:
        # Try to use OpenCV's built-in face detector
        detector = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
        print("✅ Haar Cascade face detector loaded successfully")
        return detector, 'haar'
    except Exception as e:
        print(f"❌ Error loading face detector: {e}")
        return None, None

def detect_faces_opencv(image, detector, detector_type='haar'):
    """
    Detect faces using OpenCV
    
    Args:
        image: Input image
        detector: Face detector object
        detector_type: Type of detector ('haar' or 'dnn')
    
    Returns:
        faces: List of face bounding boxes
        annotated_image: Image with face boxes drawn
    """
    if detector is None:
        return [], image.copy()
    
    # Convert to grayscale for Haar cascade
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    if detector_type == 'haar':
        # Detect faces using Haar cascade
        faces = detector.detectMultiScale(
            gray,
            scaleFactor=1.1,
            minNeighbors=5,
            minSize=(30, 30)
        )
    
    # Draw bounding boxes
    annotated_image = image.copy()
    for (x, y, w, h) in faces:
        cv2.rectangle(annotated_image, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(annotated_image, 'Face', (x, y - 10), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
    
    return faces, annotated_image

# Load face detector
face_detector, detector_type = load_face_detector()


In [ ]:
# Test face detection on sample images
def test_face_detection():
    """Test face detection on available images"""
    
    # List of test images
    test_images = [
        'images/face.jpeg',
        'images/person.jpg',
        'images/person_1.jpg',
        'images/person_2.jpg'
    ]
    
    plt.figure(figsize=(15, 10))
    
    for i, img_path in enumerate(test_images):
        if os.path.exists(img_path):
            # Load image
            img = cv2.imread(img_path)
            if img is not None:
                img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                
                # Detect faces
                faces, annotated_img = detect_faces_opencv(img, face_detector, detector_type)
                annotated_img_rgb = cv2.cvtColor(annotated_img, cv2.COLOR_BGR2RGB)
                
                # Display results
                plt.subplot(2, len(test_images), i + 1)
                plt.imshow(img_rgb)
                plt.title(f'Original: {Path(img_path).name}')
                plt.axis('off')
                
                plt.subplot(2, len(test_images), i + 1 + len(test_images))
                plt.imshow(annotated_img_rgb)
                plt.title(f'Detected: {len(faces)} face(s)')
                plt.axis('off')
                
                print(f"✅ {img_path}: {len(faces)} face(s) detected")
            else:
                print(f"❌ Could not load {img_path}")
        else:
            print(f"❌ File not found: {img_path}")
    
    plt.tight_layout()
    plt.show()

# Run face detection test
test_face_detection()


## 4. Facial Landmark Detection with Dlib

Facial landmarks help identify key points on the face like eyes, nose, mouth, and jaw contours.


In [ ]:
def detect_facial_landmarks(image, face_locations):
    """
    Detect facial landmarks using face_recognition library
    
    Args:
        image: Input image
        face_locations: List of face bounding boxes
    
    Returns:
        landmarks_list: List of facial landmarks for each face
        annotated_image: Image with landmarks drawn
    """
    try:
        # Convert OpenCV format to face_recognition format
        rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # Convert face locations from (x,y,w,h) to (top,right,bottom,left)
        face_locations_converted = []
        for (x, y, w, h) in face_locations:
            face_locations_converted.append((y, x + w, y + h, x))
        
        # Get facial landmarks
        landmarks_list = face_recognition.face_landmarks(rgb_image, face_locations_converted)
        
        # Draw landmarks on image
        annotated_image = image.copy()
        
        for landmarks in landmarks_list:
            # Draw different facial features with different colors
            colors = {
                'chin': (255, 0, 0),           # Red
                'left_eyebrow': (0, 255, 0),   # Green
                'right_eyebrow': (0, 255, 0),  # Green
                'nose_bridge': (0, 0, 255),    # Blue
                'nose_tip': (0, 0, 255),       # Blue
                'left_eye': (255, 255, 0),     # Cyan
                'right_eye': (255, 255, 0),    # Cyan
                'top_lip': (255, 0, 255),      # Magenta
                'bottom_lip': (255, 0, 255)    # Magenta
            }
            
            for feature, points in landmarks.items():
                color = colors.get(feature, (255, 255, 255))
                for point in points:
                    cv2.circle(annotated_image, point, 2, color, -1)
        
        return landmarks_list, annotated_image
        
    except Exception as e:
        print(f"❌ Error in landmark detection: {e}")
        return [], image.copy()

def draw_landmark_connections(image, landmarks_list):
    """
    Draw connections between landmark points to show facial structure
    """
    annotated_image = image.copy()
    
    for landmarks in landmarks_list:
        # Draw connections for facial features
        feature_connections = [
            ('chin', (255, 0, 0)),
            ('left_eyebrow', (0, 255, 0)),
            ('right_eyebrow', (0, 255, 0)),
            ('left_eye', (255, 255, 0)),
            ('right_eye', (255, 255, 0)),
            ('top_lip', (255, 0, 255)),
            ('bottom_lip', (255, 0, 255))
        ]
        
        for feature, color in feature_connections:
            if feature in landmarks:
                points = landmarks[feature]
                for i in range(len(points) - 1):
                    cv2.line(annotated_image, points[i], points[i + 1], color, 1)
                
                # Close the loop for eyes and lips
                if feature in ['left_eye', 'right_eye', 'top_lip', 'bottom_lip']:
                    cv2.line(annotated_image, points[-1], points[0], color, 1)
    
    return annotated_image

print("✅ Facial landmark detection functions defined")


In [ ]:
# Test facial landmark detection
def test_landmark_detection():
    """Test facial landmark detection on sample images"""
    
    test_images = [
        'images/face.jpeg',
        'images/person.jpg'
    ]
    
    for img_path in test_images:
        if os.path.exists(img_path):
            # Load image
            img = cv2.imread(img_path)
            if img is not None:
                print(f"\n🔍 Processing {img_path}...")
                
                # First detect faces
                faces, _ = detect_faces_opencv(img, face_detector, detector_type)
                print(f"   Found {len(faces)} face(s)")
                
                if len(faces) > 0:
                    # Detect landmarks
                    landmarks_list, landmarks_img = detect_facial_landmarks(img, faces)
                    connections_img = draw_landmark_connections(img, landmarks_list)
                    
                    # Display results
                    plt.figure(figsize=(15, 5))
                    
                    # Original image
                    plt.subplot(1, 3, 1)
                    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
                    plt.title('Original Image')
                    plt.axis('off')
                    
                    # Landmarks points
                    plt.subplot(1, 3, 2)
                    plt.imshow(cv2.cvtColor(landmarks_img, cv2.COLOR_BGR2RGB))
                    plt.title(f'Facial Landmarks ({len(landmarks_list)} face(s))')
                    plt.axis('off')
                    
                    # Landmark connections
                    plt.subplot(1, 3, 3)
                    plt.imshow(cv2.cvtColor(connections_img, cv2.COLOR_BGR2RGB))
                    plt.title('Landmark Connections')
                    plt.axis('off')
                    
                    plt.tight_layout()
                    plt.show()
                    
                    # Print landmark details
                    for i, landmarks in enumerate(landmarks_list):
                        print(f"   Face {i+1}: {len(landmarks)} landmark groups detected")
                        for feature, points in landmarks.items():
                            print(f"     - {feature}: {len(points)} points")
                else:
                    print("   No faces detected for landmark analysis")
            else:
                print(f"❌ Could not load {img_path}")
        else:
            print(f"❌ File not found: {img_path}")

# Run landmark detection test
test_landmark_detection()


## 5. Face Recognition using Face Embeddings

Face recognition involves creating unique embeddings (feature vectors) for each face and comparing them to identify individuals.


In [ ]:
def create_face_encodings(image_path, name):
    """
    Create face encodings for a person from their image
    
    Args:
        image_path: Path to the person's image
        name: Name of the person
    
    Returns:
        Dictionary with name and encoding
    """
    try:
        # Load image
        image = face_recognition.load_image_file(image_path)
        
        # Get face encodings
        encodings = face_recognition.face_encodings(image)
        
        if len(encodings) > 0:
            print(f"✅ Created encoding for {name} from {image_path}")
            return {
                'name': name,
                'encoding': encodings[0],  # Take first face found
                'image_path': image_path
            }
        else:
            print(f"❌ No face found in {image_path} for {name}")
            return None
            
    except Exception as e:
        print(f"❌ Error creating encoding for {name}: {e}")
        return None

def recognize_faces_in_image(image_path, known_encodings, tolerance=0.6):
    """
    Recognize faces in an image by comparing with known encodings
    
    Args:
        image_path: Path to image to analyze
        known_encodings: List of known face encodings
        tolerance: Face matching tolerance (lower = stricter)
    
    Returns:
        Results dictionary with matches and annotated image
    """
    try:
        # Load image
        image = face_recognition.load_image_file(image_path)
        
        # Find face locations and encodings
        face_locations = face_recognition.face_locations(image)
        face_encodings = face_recognition.face_encodings(image, face_locations)
        
        # Convert to OpenCV format for drawing
        cv_image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        results = []
        
        # Compare each found face with known faces
        for i, face_encoding in enumerate(face_encodings):
            matches = []
            distances = []
            
            for known_face in known_encodings:
                if known_face is not None:
                    # Compare faces
                    match = face_recognition.compare_faces([known_face['encoding']], face_encoding, tolerance=tolerance)
                    distance = face_recognition.face_distance([known_face['encoding']], face_encoding)[0]
                    
                    matches.append(match[0])
                    distances.append(distance)
            
            # Find best match
            if any(matches):
                best_match_index = distances.index(min(distances))
                if matches[best_match_index]:
                    name = known_encodings[best_match_index]['name']
                    confidence = 1 - distances[best_match_index]
                else:
                    name = "Unknown"
                    confidence = 0
            else:
                name = "Unknown"
                confidence = 0
            
            # Draw bounding box and label
            top, right, bottom, left = face_locations[i]
            
            # Choose color based on recognition
            color = (0, 255, 0) if name != "Unknown" else (0, 0, 255)
            
            # Draw rectangle and label
            cv2.rectangle(cv_image, (left, top), (right, bottom), color, 2)
            cv2.rectangle(cv_image, (left, bottom - 35), (right, bottom), color, cv2.FILLED)
            
            # Add text
            label = f"{name} ({confidence:.2f})" if name != "Unknown" else "Unknown"
            cv2.putText(cv_image, label, (left + 6, bottom - 6), 
                       cv2.FONT_HERSHEY_DUPLEX, 0.6, (255, 255, 255), 1)
            
            results.append({
                'name': name,
                'confidence': confidence,
                'location': (top, right, bottom, left)
            })
        
        return {
            'results': results,
            'annotated_image': cv_image,
            'total_faces': len(face_locations)
        }
        
    except Exception as e:
        print(f"❌ Error in face recognition: {e}")
        return None

print("✅ Face recognition functions defined")


In [ ]:
# Create a face recognition system
def setup_face_recognition_system():
    """
    Set up a face recognition system with known faces
    """
    print("🔧 Setting up Face Recognition System...")
    
    # Define known people (you can add more)
    known_people = [
        {'name': 'Person_1', 'image': 'images/person_1.jpg'},
        {'name': 'Person_2', 'image': 'images/person_2.jpg'},
    ]
    
    # Create encodings for known people
    known_encodings = []
    
    for person in known_people:
        if os.path.exists(person['image']):
            encoding = create_face_encodings(person['image'], person['name'])
            if encoding is not None:
                known_encodings.append(encoding)
        else:
            print(f"❌ Image not found: {person['image']}")
    
    print(f"✅ Face recognition system ready with {len(known_encodings)} known faces")
    return known_encodings

def test_face_recognition():
    """
    Test face recognition on sample images
    """
    # Setup the recognition system
    known_encodings = setup_face_recognition_system()
    
    if len(known_encodings) == 0:
        print("❌ No known faces available for recognition")
        return
    
    # Test images
    test_images = [
        'images/face.jpeg',
        'images/person.jpg',
        'images/person_1.jpg',
        'images/person_2.jpg'
    ]
    
    plt.figure(figsize=(15, 10))
    
    valid_results = 0
    
    for i, img_path in enumerate(test_images):
        if os.path.exists(img_path):
            print(f"\n🔍 Analyzing {img_path}...")
            
            # Perform face recognition
            result = recognize_faces_in_image(img_path, known_encodings)
            
            if result is not None:
                # Display results
                plt.subplot(2, len(test_images), valid_results + 1)
                original_img = cv2.imread(img_path)
                plt.imshow(cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB))
                plt.title(f'Original: {Path(img_path).name}')
                plt.axis('off')
                
                plt.subplot(2, len(test_images), valid_results + 1 + len(test_images))
                plt.imshow(cv2.cvtColor(result['annotated_image'], cv2.COLOR_BGR2RGB))
                plt.title(f'Recognition: {result["total_faces"]} face(s)')
                plt.axis('off')
                
                # Print recognition results
                for j, face_result in enumerate(result['results']):
                    print(f"   Face {j+1}: {face_result['name']} (confidence: {face_result['confidence']:.3f})")
                
                valid_results += 1
            else:
                print(f"   ❌ Could not process {img_path}")
        else:
            print(f"❌ File not found: {img_path}")
    
    if valid_results > 0:
        plt.tight_layout()
        plt.show()
    else:
        print("❌ No valid results to display")

# Run face recognition test
test_face_recognition()


## 6. Real-Time Face Recognition

Implement real-time face recognition using webcam input.
